<a href="https://colab.research.google.com/github/Andrej-Ilin/practice/blob/main/Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.2 Стекинг

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import clone

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from matplotlib import pyplot as plt

%matplotlib inline

In [10]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz', sep=',', header=None)[:10000]

In [11]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [12]:
df.shape

(10000, 55)

In [13]:
features = list(range(0, 54))
target = 54

df = df[(df[target] == 1) | (df[target] == 2)]

In [14]:
cover_train, cover_test = train_test_split(df, test_size=0.5)

cover_X_train, cover_y_train = cover_train[features], cover_train[target]
cover_X_test, cover_y_test = cover_test[features], cover_test[target]

In [15]:
scaler = StandardScaler()
cover_X_train = scaler.fit_transform(cover_X_train)
cover_X_test = scaler.transform(cover_X_test)

In [16]:
n_classes = len(np.unique(cover_y_train)) # 
X_meta_train = np.zeros((len(cover_X_train), n_classes), dtype=np.float32)

In [17]:
X_meta_train

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

Stacking — еще один способ объединить несколько алгоритмов в один, который часто используется как в решении реальных задач из промышленной сферы, так и в конкурсах на платформах вроде Kaggle.  
Подход использует понятие _базовых классификаторов_, каждый из которых независимо обучается на некотором (возможно одном и том же) множестве признаков, а также _мета-классификатора_, использующего предсказания базовых классификаторов как признаки.

Для избежания переобучения будем разбивать обучающую выборку на фолды.  
Например, фолды при разбиении на три части:  
``==*``  
``=*=``  
``*==``  

Это требуется для того, чтобы получить новые признаки (ответы алгоритмов на первом уровне) на всей обучающей выборке, т.е. ответы алгоритма на тех объектах, которые не были использованы во время обучения. В примере выше мы будем использовать ответы алгоритма, полученные на объектах звездочках. _Важно_: на каждом фолде мы обучаем алгоритм заново.

In [18]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features using the classifier.
    
    :arg clf: scikit-learn classifier
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

In [19]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
    """
    Generates metafeatures using a list of classifiers.
    
    :arg classifiers: list of scikit-learn classifiers
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

In [20]:
np.random.seed(42)

In [21]:
clf = GradientBoostingClassifier(n_estimators=300)
clf.fit(cover_X_train, cover_y_train)

accuracy_score(clf.predict(cover_X_test), cover_y_test)

0.7822410147991543

In [22]:
cv = KFold(n_splits=10, shuffle=True)

stacked_features_train, stacked_features_test = generate_meta_features([
    LogisticRegression(C=0.001, penalty='l1', solver='liblinear', max_iter=5000),
    LogisticRegression(C=0.001, penalty='l2', solver='liblinear', max_iter=5000),  
    RandomForestClassifier(n_estimators=300, n_jobs=-1),
    GradientBoostingClassifier(n_estimators=300)
], cover_X_train, cover_X_test, cover_y_train.values, cv)

100%|██████████| 4/4 [00:24<00:00,  6.21s/it]


In [23]:
total_features_train = np.hstack([cover_X_train, stacked_features_train])
total_features_test = np.hstack([cover_X_test, stacked_features_test])

In [24]:
np.random.seed(42)
clf = LogisticRegression(penalty='none', solver='lbfgs')
clf.fit(stacked_features_train, cover_y_train)
accuracy_score(clf.predict(stacked_features_test), cover_y_test)

0.7998590556730092

Все задания выполняются на основе датасета:

In [25]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

In [26]:
dataset = load_digits()
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

В скринкасте мы разобрали схему генерации признаков в стекинге, когда для тестовой выборки алгоритм заново переобучался на всей тренировочной выборке. Реализуйте схему, когда вместо этого производится агрегация ответов всех обученных на фолдах классификаторов на тестовой выборке при помощи усреднения.

Логика решения:

*1) Создадим X_meta_test, заполним его нулями (по аналогии с X_meta_train);

2) Далее на каждом шаге, где мы обучаем folded_clf.fit (X_fold_train, y_fold_train) и его предсказания на X_fold_predict запихиваем в X_meta_train[predict_fold_index] добавим еще одну строку, где в X_meta_test будем добавлять предсказания вероятностей folded_clf на X_test. Их можно сразу складывать друг с другом или сохранить много массивов, тогда в конце их нужно будет все сложить, а потом делить на количество сплитов (количество массивов равно количеству сплитов в кросс - валидации);

3) После цикла останется только усреднить все эти массивы, это и будет наш X_meta_test.*

In [27]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора. 
    Они являются вероятностями классов при решении задачи многоклассовой классификации.
    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)
    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
    n_classes = len(np.unique(y_train)) # сколько уник. элементов класса
    # Создадим матрицу из нулей размером количества уник элементов класса.
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32) 
    X_meta_tests_array = [] # Сюда будем закидывать массивы результатов предсказаний 
    splits = 0
    """cv - разбиваем на 10 частей и по очериди их меняем местами в train и test"""
    for train_fold_index, predict_fold_index in cv.split(X_train):
        """ создаем пустой массив для результата предсказаний"""
        n_classes = len(np.unique(y_test)) 
        X_meta_test = np.zeros((len(X_test), n_classes), dtype=np.float32)
        splits += 1 
        """разбиение происходит по индексам"""
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
        X_meta_tests_array.append(folded_clf.predict_proba(X_test))
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    X_meta_test = sum(X_meta_tests_array) / splits
    return X_meta_train, X_meta_test

Для следующих заданий используйте этот код:

In [28]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [29]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

In [30]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred, average='macro'), 6)

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

* логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимое количество итераций — 2000
* логистическая регрессия с L2-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — multinomial, максимальное допустимое количество итераций — 2000
* случайный лес из 300 деревьев
* градиентный бустинг из 200 деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [38]:
lg_reg1 = LogisticRegression(penalty='l1', C=0.001, solver='saga', multi_class='ovr', max_iter=2000, n_jobs=-1, random_state=42)
lg_reg2 = LogisticRegression(penalty='l2', C=0.001, solver='saga', multi_class='multinomial', max_iter=2000, n_jobs=-1, random_state=42)
r_f = RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)
gr_b = GradientBoostingClassifier(n_estimators=200, random_state=42)

In [39]:
stacked_features_train, stacked_features_test = generate_meta_features([
                                            lg_reg1, 
                                            lg_reg2, 
                                            r_f, 
                                            gr_b
                                            ], 
                                           X_train, X_test, y_train, cv)


100%|██████████| 4/4 [02:20<00:00, 35.17s/it]


Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.
Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [41]:
meta_lr = LogisticRegression(solver='lbfgs', multi_class='auto', random_state=42)
meta_lr.fit(stacked_features_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [42]:
compute_metric(meta_lr, stacked_features_train, y_train, stacked_features_test)

0.980848

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
* случайный лес из 300 деревьев
* случайный лес из 200 экстремальных деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [44]:
r_f1 = RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)
r_f2 = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)
stacked_features_train, stacked_features_test = generate_meta_features([ 
                                            r_f1,                            
                                            r_f2,                                             
                                            ], 
                                           X_train, X_test, y_train, cv)


100%|██████████| 2/2 [00:20<00:00, 10.01s/it]


In [45]:
meta_lr = LogisticRegression(solver='lbfgs', multi_class='auto', random_state=42)
meta_lr.fit(stacked_features_train, y_train)
compute_metric(meta_lr, stacked_features_train, y_train, stacked_features_test)

0.978682

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

* метод ближайшего соседа (k-NN) со стандартными параметрами
* случайный лес из 300 экстремальных деревьев

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [49]:
knn = KNeighborsClassifier(n_jobs=-1)
r_f_e = ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42)
stacked_features_train, stacked_features_test = generate_meta_features([ 
                                            knn,                            
                                            r_f_e,                                             
                                            ], 
                                           X_train, X_test, y_train, cv)


100%|██████████| 2/2 [00:10<00:00,  5.12s/it]


In [50]:
meta_lr = LogisticRegression(solver='lbfgs', multi_class='auto', random_state=42)
meta_lr.fit(stacked_features_train, y_train)
compute_metric(meta_lr, stacked_features_train, y_train, stacked_features_test)

0.987798

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:

* логистическая регрессия с L1-регуляризацией, C=0.001, солвер — 'saga', схема работы мультиклассовой классификации — one-vs-rest, максимальное допустимоей количество итераций — 2000
* метод ближайшего соседа со стандартными параметрами
* случайный лес из 300 экстремальных деревьев
* AdaBoost со стандартными параметрами

Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [51]:
lg_reg = LogisticRegression(penalty='l1', C=0.001, solver='saga', multi_class='ovr', max_iter=2000, n_jobs=-1, random_state=42)
knn = KNeighborsClassifier(n_jobs=-1)
r_f_e = ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1)
ada = AdaBoostClassifier(random_state=42)
stacked_features_train, stacked_features_test = generate_meta_features([                                                                         
                                            lg_reg,
                                            knn,
                                            r_f_e,
                                            ada
                                            ], 
                                           X_train, X_test, y_train, cv)


100%|██████████| 4/4 [00:49<00:00, 12.26s/it]


In [52]:
meta_lr = LogisticRegression(solver='lbfgs', multi_class='auto', random_state=42)
meta_lr.fit(stacked_features_train, y_train)
compute_metric(meta_lr, stacked_features_train, y_train, stacked_features_test)

0.987798

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
* случайный лес из 300 деревьев
* случайный лес из 300 экстремальных деревьев

Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).

Для корректной работы необходимо подправить код в функции compute_meta_feature. Как мета-алгоритм используйте логистическую регрессию без регуляризации со схемой работы мультиклассовой классификации — auto и солвером 'lbfgs'.

Посчитайте качество при помощи передачи новых признаков в функцию compute_metric. Количество фолдов = 10

In [54]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)

    splits = cv.split(X_train, y_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)
    
    return X_meta_train, X_meta_test

In [59]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
r_f = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
r_f_e = ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1)
stacked_features_train, stacked_features_test = generate_meta_features([
                                            r_f,
                                            r_f_e
                                            ], 
                                           X_train, X_test, y_train, cv)



  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:06<00:06,  6.34s/it]

100%|██████████| 2/2 [00:11<00:00,  5.54s/it]


In [62]:
meta_lr = GradientBoostingClassifier(random_state=42)
meta_lr.fit(stacked_features_train, y_train)
compute_metric(meta_lr, stacked_features_train, y_train, stacked_features_test)

0.987407

Используйте функцию generate_meta_features для стекинга следующих алгоритмов:
* случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
* случайный лес из 300 экстремальных деревьев

Для генерации фолдов используйте класс StratifiedKFold, который позволяет делать так называемые стратифицированные разбиения (в каждом фолде будет одинаковое соотношение классов).

Для генерации фолдов используйте класс StratifiedKFold и поправленный Вами ранее код в функции compute_meta_feature.
Выполните разбиение на 3 фолда.

Как мета-алгортм используйте случайный лес из 100 экстремальных деревьев. Посчитайте качество при помощи передачи новых признаков в функцию compute_metric.

In [64]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
r_f = RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=24, random_state=42, n_jobs=-1)
r_f_e = ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1)
stacked_features_train, stacked_features_test = generate_meta_features([
                                            r_f,
                                            r_f_e
                                            ], 
                                           X_train, X_test, y_train, cv)




  0%|          | 0/2 [00:00<?, ?it/s]


 50%|█████     | 1/2 [00:03<00:03,  3.91s/it]


100%|██████████| 2/2 [00:06<00:00,  3.46s/it]


In [65]:
meta_lr = RandomForestClassifier(n_estimators=100, random_state=42)
meta_lr.fit(stacked_features_train, y_train)
compute_metric(meta_lr, stacked_features_train, y_train, stacked_features_test)

0.983967

Обучите на тренировочной выборке следующие алгоритмы:
* случайный лес из 300 деревьев, критерий Джини, максимальная глубина — 24
* случайный лес из 300 экстремальных деревьев
* логистическую регрессию со стандартными параметрами

Усредните их ответы на тестовой выборке методом сложения предсказаний и затем взятия функции argmax: answer = (prediction1 + prediction2 + prediction3).argmax(axis = 1).

Посчитайте качество, аналогично функции compute_metric (F1-score с макро-усреднением, округленный до 6 знака).

In [75]:
r_f = RandomForestClassifier(n_estimators=300, criterion='gini', random_state=42, n_jobs=-1)
r_f_e = ExtraTreesClassifier(n_estimators=300, random_state=42, n_jobs=-1)
lg_reg = LogisticRegression()

def comptue_pipline(classifiers, X_train, X_test, y_train):
  predict_array = []
  for clf in tqdm(classifiers):
    clf.fit(X_train, y_train)
    array = clf.predict_proba(X_test)
    predict_array.append(array)
  return predict_array
predict_array = comptue_pipline([r_f, r_f_e, lg_reg], X_train, X_test, y_train)

y_test_pred = np.sum(predict_array, axis=0).argmax(axis=1)
np.round(f1_score(y_test, y_test_pred, average='macro'), 6)




  0%|          | 0/3 [00:00<?, ?it/s]


 33%|███▎      | 1/3 [00:01<00:02,  1.13s/it]


 67%|██████▋   | 2/3 [00:01<00:01,  1.04s/it]/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)



100%|██████████| 3/3 [00:02<00:00,  1.34it/s]


0.976259